In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../csv/BND.csv", index_col="Date")
df.head()

In [ ]:
price_df = df.loc[:, ["Adj Close"]]
price_df.head(1)

In [ ]:
## 이동 평균선 파생변수 생성 : 데이터양이 20개의 평균을 구해서 컬럼에 대입
price_df["center"] = price_df["Adj Close"].rolling(20).mean()
price_df.iloc[18:25]

In [ ]:
## 상단 밴드 파생변수 생성 ( 이동 평균선 + 2 * 표준편차 )
price_df['ub'] = price_df["center"] + 2 * price_df["Adj Close"].rolling(20).std()
price_df.iloc[18:25]

In [ ]:
## 하단 밴드 파생변수 생성 ( 이동 평균선 - 2 * 표준편차 )
price_df["lb"] = price_df["center"] - 2 * price_df["Adj Close"].rolling(20).std()
price_df.iloc[18:25]

In [ ]:
## 시작 시간 
start_time = "2010-01-02"
price_df = price_df.loc[start_time:]
price_df

In [ ]:
## trade 파생변수 값들은 비어있는 값
price_df["trade"] = ""
price_df.head(1)

In [15]:
## 구매 내역 데이터를 삽입
## 조건 : 상단 밴드보다 종가가 높은 경우
##          구매 상태 이면
##              매도 trade = ""
##          구매 상태가 아니면
##              아무 행동도 하지 않는다 trade = ""
##       하단 밴드보다 종가가 낮은 경우
##          구매 상태이면 
##              구매 상태 유지 trade = "buy"
##          구매 상태가 아니면
##              구매 trade = "buy"
##       상단 밴드와 하단 밴드 사이에 종가가 존재하면 
##          구매 상태이면 
##              현 상태 유지 trade = "buy"
##          구매 상태가 아니면
##              현 상태 유지 trade = ""
for i in price_df.index:
    if price_df.loc[i, "Adj Close"] > price_df.loc[i, "ub"]:
        price_df.loc[i, "trade"] = ""
    elif price_df.loc[i, "lb"] > price_df.loc[i, "Adj Close"]:
        price_df.loc[i, "trade"] = "buy"
    elif price_df.loc[i, "ub"] >= price_df.loc[i, "Adj Close"] and \
         price_df.loc[i, "Adj Close"] >= price_df.loc[i, "lb"]:
        if price_df.shift(1).loc[i, "trade"] == "buy":
            price_df.loc[i, "trade"] = "buy"
        else :
            price_df.loc[i, "trade"] = ""

price_df.value_counts("trade")
             

c:\Users\moons\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


trade
       1568
buy     816
dtype: int64

In [19]:
price_df.query("trade == 'buy'")

,Adj Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-03-25,61.205368,61.569890,61.819734,61.320047,buy,1.0,1.000000
2010-03-26,61.337158,61.560745,61.830838,61.290653,buy,1.0,1.000000
2010-03-29,61.205368,61.541755,61.854636,61.228875,buy,1.0,1.000000
2010-03-30,61.329365,61.526641,61.850257,61.203025,buy,1.0,1.000000
2010-03-31,61.383640,61.513850,61.838911,61.188789,buy,1.0,1.000000
...,...,...,...,...,...,...,...
2019-03-01,78.634193,78.905679,79.169995,78.641363,buy,1.0,1.070881
2019-03-04,78.772850,78.907841,79.166744,78.648939,buy,1.0,1.070881
2019-03-05,78.832283,78.916434,79.151029,78.681839,buy,1.0,1.070881


In [ ]:
## 손익 계산
rtn = 1.0
price_df["return"] = 1
buy = 0.0
sell = 0.0
for i in price_df.index:
    ## 구매하는 경우
    if price_df.loc[i, "trade"] == "buy" and price_df.shift(1).loc[i, "trade"] == "":
        buy = price_df.loc[i, "Adj Close"]
        print("진입일 : ", i, "진입 가격 : ", buy)
    ## 매도하는 경우
    elif price_df.loc[i, "trade"] == "" and price_df.shift(1).loc[i, "trade"] == "buy":
        sell = price_df.loc[i, "Adj Close"]
        rtn = (sell - buy) / buy + 1 # 손익 계산
        price_df.loc[i, "return"] = rtn
        print("청산일 : ", i, "진입 가격 : ", buy, "청산 가격 : ", sell,\
             "| return : ", round(rtn, 4))
    
    ## buy, sell 변수 초기화
    if price_df.loc[i, "trade"] == "":
        buy = 0.0
        sell = 0.0
    
acc_rtn = 1.0
for i in price_df.index:
    rtn = price_df.loc[i, "return"]
    acc_rtn = acc_rtn * rtn
    price_df.loc[i, "acc_rtn"] = acc_rtn

print("누적 수익률 : ", round(acc_rtn, 4))

### 볼린저밴드 모듈 생성
1. bollinger 폴더를 생성
2. ```__init__.py``` 파일을 생성
3. 클래스 생성 
4. 데이터프레임, 기준이되는 컬럼 ```__init__``` 함수에서 인자로 받아오는 함수
5. 이동 평균선, 상단밴드, 하단밴드, trade = "" 생성 함수 생성
6. 시작시간은 인자값으로 받아와서 데이터의 개수 필터링
7. 구매 상태를 삽입 함수 생성
8. 손익 계산 함수 생성

In [20]:
def func_1(start_time):     ## start_time : 매개변수
    return start_time

func_1('2020-01-01')        ## 2020-01-01 : 인자

'2020-01-01'